In [5]:
import pandas as pd

In [15]:
import numpy as np

In [24]:
import nltk

In [126]:
from nltk.stem import PorterStemmer

In [31]:
from nltk.tokenize import word_tokenize

In [162]:
from nltk.corpus import stopwords

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
from sklearn.preprocessing import OneHotEncoder

In [59]:
from sklearn.linear_model import LogisticRegression

In [87]:
from sklearn.model_selection import train_test_split

In [163]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [164]:
stop_words = stopwords.words('english')

In [1]:
# textos = []
# sentimentos = []

In [2]:
# arquivo = open("../../../dados/nlp/news_sentiment_analysis.csv", "r", encoding="utf-8")
# arquivo.readline()   # Descartando a primeira linha (header)
# linha = arquivo.readline()
# while linha != "":
#     colunas = linha.split(",")
#     texto = colunas[3]
#     sentimento = colunas[6]
#     textos.append(texto)
#     sentimentos.append(sentimento)
#     linha = arquivo.readline()
# arquivo.close()
    

In [6]:
df_news = pd.read_csv("../../../dados/nlp/news_sentiment_analysis.csv", sep=",", quotechar='"', encoding="utf-8")

In [12]:
df_news = df_news[df_news['Sentiment'] != "neutral"]

In [20]:
df_news["Sentiment_number"] = df_news['Sentiment'].apply(lambda x: 0 if x == 'negative' else 1)

C:\Users\anton\AppData\Local\Temp\ipykernel_14212\330576338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news["Sentiment_number"] = df_news['Sentiment'].apply(lambda x: 0 if x == 'negative' else 1).copy()


In [22]:
# df_news["description_lower"] = df.news['Description'].apply(lambda x: x.lower())
df_news["description_lower"] = df_news['Description'].str.lower()

C:\Users\anton\AppData\Local\Temp\ipykernel_14212\2738751418.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news["description_lower"] = df_news['Description'].str.lower()


In [129]:
ps = PorterStemmer()

In [176]:
def remove_stopwords( texto ):
    tokens = word_tokenize( texto )
    nova_lista = []
    for token in tokens:
        if token not in stop_words:
            # nova_lista.append(ps.stem(token))
            nova_lista.append(token)
    return " ".join(nova_lista)
    

In [177]:
df_news["description_lower"][0]

'st. george — kaitlyn larson, a first-year teacher at pine view high school, recently received the best in state award for a business leadership course. larson’s students in grades 10-12 had the highest passing rate on the youscience exam during the spring and fall semesters of 2023. the youscience exam is similar to final exams [&#8230;]'

In [178]:
stemmer(df_news["description_lower"][0])

'st. georg — kaitlyn larson , a first-year teacher at pine view high school , recent receiv the best in state award for a busi leadership cours . larson ’ s student in grade 10-12 had the highest pass rate on the youscienc exam dure the spring and fall semest of 2023. the youscienc exam is similar to final exam [ & # 8230 ; ]'

In [179]:
df_news["description_stem"] = df_news["description_lower"].apply(lambda x: remove_stopwords(x))

C:\Users\anton\AppData\Local\Temp\ipykernel_14212\521870535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news["description_stem"] = df_news["description_lower"].apply(lambda x: remove_stopwords(x))


In [180]:
df_news["description_stem"]

0       st. george — kaitlyn larson , first-year teach...
2       ( marketscreener.com ) billionaire elon musk d...
3       ( marketscreener.com ) u.s. trade regulator fr...
4       4.5 million households u.s. solar panels homes...
5       ( marketscreener.com ) billionaire investor ma...
                              ...                        
3493    qrg capital management inc. trimmed position s...
3495    qrg capital management inc. increased stake sh...
3496    qrg capital management inc. bought new positio...
3497    qrg capital management inc. boosted stake akam...
3499    qrg capital management inc. reduced holdings s...
Name: description_stem, Length: 2711, dtype: object

In [181]:
#vetorizador = CountVectorizer(max_features=2000)
vetorizador = CountVectorizer()

In [182]:
bow_x = vetorizador.fit_transform(df_news["description_stem"])

In [183]:
vocabulario = vetorizador.get_feature_names_out()
len(vocabulario)

16348

In [184]:
x_train, x_test, y_train, y_test = train_test_split(bow_x, df_news["Sentiment_number"], random_state=50)

In [185]:
one_hot_train = pd.DataFrame.sparse.from_spmatrix(x_train, columns=vocabulario)
one_hot_test = pd.DataFrame.sparse.from_spmatrix(x_test, columns=vocabulario)

In [186]:
model = LogisticRegression()

In [187]:
model.fit(one_hot_train, y_train)

LogisticRegression()

In [188]:
model.score(one_hot_test, y_test)

0.9100294985250738

### Score com todas palavras do vocabulario ==> 0.9129793510324484
### Score com 2000 palavras no vocabulario ==> 0.9174041297935103

In [76]:
novo_vetorizador = CountVectorizer(vocabulary=vocabulario)

In [83]:
bow_x_predict = novo_vetorizador.fit_transform(["The Baltimore Business Journal has hired Ben Terzi as its research editor. Terzi manages various databases to produce weekly Lists, the Book of Lists and other research projects for the Baltimore Business Journal. Prior to joining the Business Journal, he covered community news for The Dundalk Eagle and Avenue News. Born and raised in Baltimore, Terzi is"])

In [84]:
x_predict = pd.DataFrame.sparse.from_spmatrix(bow_x_predict, columns=vocabulario)

In [85]:
model.predict(x_predict)

array([1], dtype=int64)